# Test for vectorized speed up

In [1]:
import sys
import os
sys.path.append('./src')
os.chdir(os.path.dirname(sys.path[0]))



# data Analysis 
import geopandas as gpd
import pandas as pd
import numpy as np



# plot
import matplotlib.pyplot as plt
import gc

# output
from output_image import write_output, analyze_model_result, output_to_csv


# multiprocessing
import multiprocessing
import brun


# model
from school_model import School

# spreadsheets
# import spreadsheets

#config
import configparser
import warnings
warnings.filterwarnings("ignore")

In [2]:
map_path = "../layouts/schoollayout1/schoollayout1.shp"
schedule_path = "../schedule_data/day_schedule.csv" #small schedule has only 5 steps per day for testing purposes
#full day schedule should be "day_schedule.csv" 
schedule_steps = 90 # full day_schedule steps should be 90


# if you wish to use google sheet parameter input, you'll have to follow instructions for getting credentials of the sheet api
# follow the first two steps in:
# https://developers.google.com/sheets/api/quickstart/python
# and save your credentials in the src folder as listed below
SHEET_URL = 'https://docs.google.com/spreadsheets/d/1Quyyey5B_kdQK1_OU0OkIDGZE27tuUfjZ6hCsV99-sM'
credentials = '../src/credentials.json'


# two types of parameter setups available for batchrunner
# pre-setup for fixed/variable parameter dictionaries (consistant with mesa batchrunner)
######################
grade_N = 350
KG_N = 50
preschool_N = 50
special_education_N = 10
faculty_N = 40
seat_dist = 12
mask_prob = 0.516
days = 15
max_steps = days*schedule_steps
iterations = 1
####################

In [3]:
# fixed parameters are model input parameters that are static for all batch running jobs
fixed_parameters = {}


fixed_parameters['grade_N'] = grade_N
fixed_parameters['KG_N'] = KG_N
fixed_parameters['preschool_N'] = preschool_N
fixed_parameters['special_education_N'] = special_education_N
fixed_parameters['faculty_N'] = faculty_N
fixed_parameters['seat_dist'] = 12
fixed_parameters['init_patient'] = 3
#fixed_parameters['mask_prob'] = 0.5

# same logic as fixed parameters, only these will be batch ran for all combinations 
variable_parameters = {}
variable_parameters['attend_rate'] = [0.25, 0.5, 0.75, 1]
variable_parameters['inclass_lunch'] = [True, False]
variable_parameters['mask_prob'] = [0, 0.25, 0.5, 0.75, 1]

In [4]:
# Prefix for config data
#os.chdir(os.path.dirname(sys.path[0]))
config_file_path_prefix = './config/'


# parser viz config data
viz_ini_file = 'vizparams.ini'

parser_viz = configparser.ConfigParser()
parser_viz.read(config_file_path_prefix + viz_ini_file)

default_section = parser_viz['DEFAULT_PARAMS']


# parser disease config data

disease_params_ini = 'diseaseparams.ini'
parser_dis = configparser.ConfigParser()
parser_dis.read(config_file_path_prefix + disease_params_ini)
incubation = parser_dis['INCUBATION']


# NPI config data


npi_params_ini = 'NPI.ini'
parser_npi = configparser.ConfigParser()
parser_npi.read(config_file_path_prefix + npi_params_ini)

['./config/NPI.ini']

In [ ]:
'''
close_neighbor = [0,1] #cohort
class_room = [0,1] #same room
health_status = [0,1]
infective = [0,1]
infectious_rate = # from curve
mask_bias = [1, 0.25]




'''


In [7]:
agent1 = np.array([0, 0, 0, 1, # one hot cohort 
                   0, 0, 0, 1, # one hot room
                   1, # exposed
                   1, # infective
                   0.25, #infectious rate
                   1 #not wearing mask 
                  ])


agent2 = np.array([0, 0, 1, 0, # one hot cohort 
                   0, 0, 0, 1, # one hot room
                   0, # healthy
                   0, # infective
                   0, #infectious rate
                   1 #not wearing mask 
                  ])



In [9]:
agent1

array([0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 1.  , 1.  , 1.  , 0.25,
       1.  ])

In [8]:
np.dot(agent1, agent2)

2.0

In [ ]:
class Human_Vector():
    # dummy config for data collection
    viral_load = None
    
    
    
    # UPDATE 10/16: move stats to class level
     

    
    
    
    
    # infectious curve config
    ###################################### 
    # based on gamma fit of 10000 R code points

    shape, loc, scale = (float(incubation['shape']), float(incubation['loc']), float(incubation['scale']))

    # infectious curve
    range_data = list(range(int(incubation['lower_bound']), int(incubation['upper_bound']) + 1))
    infective_df = pd.DataFrame(
        {'x': range_data,
         'gamma': list(stats.gamma.pdf(range_data, a=shape, loc=loc, scale=scale))
        }
    )
    #########################################

    
    def __init__(self, unique_id, model, x, y, room, health_status = 'healthy'):
        

        
        # disease config

        self.health_status = health_status
        prevalence = float(parser_dis['ASYMPTOMATIC_PREVALENCE']['prevalence'])
        self.asymptomatic = np.random.choice([True, False], p = [prevalence, 1-prevalence])
        self.symptoms = False
        
        # UPDATE 10/17: delay infection by 1 day to avoid infection explosion
        self.infective = False
        
        # symptom onset countdown config
        ##########################################
        # From 10000 lognorm values in R
        countdown = parser_dis['COUNTDOWN']
        shape, loc, scale =  (float(countdown['shape']), float(countdown['loc']), float(countdown['scale']))

        lognormal_dist = stats.lognorm.rvs(shape, loc, scale, size=1)

        num_days = min(np.round(lognormal_dist, 0)[0], int(countdown['upper_bound'])) # failsafe to avoid index overflow
        self.symptom_countdown = int(num_days)
        #######################################
        
        

        
        self.room = room
        self.x = self.shape.x
        self.y = self.shape.y


        
    def build(self):

 
    def update_shape(self, new_shape):
        self.shape = new_shape
        self.x = self.shape.x
        self.y = self.shape.y
        
    
    def __update(self):
        # UPDATE 10/16: reorganized things from Bailey's update
        # TODO: currently mask has no functionality other than reducing transmission distance, is this faithful?


        # mask wearing reduces droplet transmission max range
        # infection above max range is considered as aerosal transmission
        if self.mask and not (self.model.activity[self.room.schedule_id] == 'lunch'):
            neighbors = self.model.grid.get_neighbors_within_distance(self, int(parser_npi['MASKS']['infection_distance']))
        else:
            neighbors = self.model.grid.get_neighbors_within_distance(self, int(parser_npi['NO_NPI']['infection_distance']))

        
        # UPDATE 10/16: infectious has made obsolete due to infectious curve covering after symptom onset fit
        # credit Bailey Man
        '''
        if self.health_status == 'infectious':
            #TODO: sliding Distance calculation for neighbor infection
            # a scale for health status


            # Loop through infected agent's neighbors that are within 3
        
            for neighbor in neighbors:
                
                ### 
                #temp value, replace this with distance -> viral load -> infection_prob calculation eventually
                temp_prob = .857 # previous value used

                # Check class is Human                            
                if issubclass(type(neighbor), Human):

                    infective_prob = np.random.choice ([True, False], p = [temp_prob, 1 - temp_prob])
                    if infective_prob and self.__check_same_room(neighbor):
                        neighbor.health_status = 'exposed'
        '''    

                        
                        
                        
        if self.health_status == 'exposed' and self.infective:

            # normalize symptom countdown value to infectious distribution value
            # 0 being most infectious
            # either -10 or 8 is proven to be too small of a chance to infect others, thus covering asympotmatic case


            countdown_norm = min(int(incubation['upper_bound']), max(int(incubation['lower_bound']), 0 - self.symptom_countdown))
            temp_prob = self.infective_df[self.infective_df['x'] == countdown_norm]['gamma'].iloc[0]

            
            for neighbor in neighbors:

                # Check class is Human                            
                if issubclass(type(neighbor), Human):
                    if neighbor.unique_id != self.unique_id:                   
                        # TODO: update this to a more realistic scale
                        agent_distance = self.shape.distance(neighbor.shape)

                        try:
                            dist_bias = np.sqrt(min(1, 1/agent_distance))
                        except:
                            dist_bias = 1


                        # row a dice of temp_prob*dist_bias chance to expose other agent
                        infective_prob = np.random.choice ([True, False], p = [temp_prob*dist_bias, 1 - (temp_prob*dist_bias)])

                        if infective_prob and self.__check_same_room(neighbor):
                            neighbor.health_status = 'exposed'

                        
    def __check_same_room(self, other_agent):
        '''
        check if current agent and other agent is in the same room
        
        the purpose of this function is to make sure to eliminate edge cases that one agent near the wall of its room
        infects another agent in the neighboring room
        
        this is at this iteration of code only implemented for class purpose, as unique id check is way more efficient
        
        later implementation should add attribute to human agent for current room
        
            other_agent: other agent to check
            returns: boolean value for if the two agents are in the same room
        '''
        same_room = True
        if self.model.activity[self.room.schedule_id] == 'class':
            same_room = (self.room.unique_id == other_agent.room.unique_id)
        return same_room
    
    
    def __move(self, move_spread = 12, location = None):
        '''
        Checks the current location and the surrounding environment to generate a feasbile range of destination (the area
        of a circle) for agent to move to.
        The radius of the circle is determined by agent's move_factor.
        Assigns new point to override current point.
        '''   
        
        if not location:
            location = self.room
        move_spread = location.shape.intersection(self.shape.buffer(move_spread))
        minx, miny, maxx, maxy = move_spread.bounds
        while True:
            pnt = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))            
            # check if point lies in true area of polygon
            if move_spread.contains(pnt):
                self.update_shape(pnt)
                break